# Beams Pipeline Tutorial

## What software do I need for this tutorial?

Welcome to the Beams Pipeline Tutorial! Before beginning, you will need the Map Multi Tool, as well as the beam systematics files.

To get the Map Multi Tool on your computer, go to your command line or terminal and paste in:

```git clone https://github.com/CMB-S4/map_multi_tool.git```

For the beam systematics, paste in:

```git clone https://github.com/laurensaunders/beams_pipeline.git```

You will also need ```numpy```, ```scipy```, ```matplotlib```, ```math```, and ```time```, so make sure that each of those is also installed on your computer.

## Introduction to Map Multi Tool

The Map Multi Tool software is a new map-based tool for studying temperature to polarization and E$\rightarrow$B polarization leakage. The tool is useful for showing ways that CMB systematics such as crosstalk, pointing miscalibration, and optical irregularities that show up in the beams can produce this leakage.

Map Multi Tool has two sample Jupyter notebooks that let you explore some of the software's functionality--both in a crosstalk implementation and in a beams implementation. Each of these notebooks has a simple model of the instrument scenario and walks through how to find the leakage spectra.

## Beam Pipeline + Map Multi Tool

The beam pipeline is a tool that allows you to make a beam and then analyze the leakage spectra for that beam. You can use it to make a completely analytic beam simulation, or to use a real beam (from HFSS), and then analyze that beam. The beam you produce can then be used with Map Multi Tool to find the leakage spectra.

Although this tutorial has pre-filled values for the beam parameters, you can try changing the values of these parameters to see what happens with different telescope designs.

In [ ]:
# import useful packages
import numpy as np
import matplotlib.pyplot as plt
import make_beams
import mmt_tools

## Analytic Beam Simulation

We'll start by making and analyzing a completely simulated beam. Currently, this tool will let you make a beam with some ellipticity (you get to choose how much); other perturbations are currently not available, but may be in the future.

We need to define the beam parameters, and then we use ```beams_pipeline.make_beams``` to set everything up. We'll use the ```AnalyticBeam``` class here, since we want to make a completely analytic simulated beam.

In [ ]:
analytic_beam_params = {'N': 1024, # number of pixels
                        'pixel_size': 0.25/60., # size of pixels in degrees
                        'beam_fwhm': 1.5, # FWHM of the beam
                        'ellipt': 0.001, # ellipticity
                        }

abeam = make_beams.AnalyticBeam(analytic_beam_params)

```abeam``` now holds all of the information we need about our simulated beam. We can plot what the beam leakage looks like and make sure it looks as expected (we'll only add colorbars for the ones that we expect to show some leakage):

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(8,8))

qq = ax[0,0].imshow(abeam.beam_matrix['QQ']) # Q->Q leakage
ax[0,0].set_title('QQ')
plt.colorbar(qq, ax=ax[0,0])

qi = ax[1,0].imshow(abeam.beam_matrix['QI']) # Q->I leakage
ax[1,0].set_title('QI')
plt.colorbar(qi, ax=ax[1,0])

qu = ax[2,0].imshow(abeam.beam_matrix['QU']) # Q->U leakage
ax[2,0].set_title('QU')
plt.colorbar(qu, ax=ax[2,0])

iq = ax[0,1].imshow(abeam.beam_matrix['IQ']) # I->Q leakage
ax[0,1].set_title('IQ')

ii = ax[1,1].imshow(abeam.beam_matrix['II']) # I->I leakage
ax[1,1].set_title('II')
plt.colorbar(ii, ax=ax[1,1])

iu = ax[2,1].imshow(abeam.beam_matrix['IU']) # I->U leakage
ax[2,1].set_title('IU')

uq = ax[0,2].imshow(abeam.beam_matrix['UQ']) # U->Q leakage
ax[0,2].set_title('UQ')

ui = ax[1,2].imshow(abeam.beam_matrix['UI']) # U->I leakage
ax[1,2].set_title('UI')

uu = ax[2,2].imshow(abeam.beam_matrix['UU']) # U->U leakage
ax[2,2].set_title('UU')
plt.colorbar(uu, ax=ax[2,2])

plt.show()

Next, we would like to see the leakage spectra. We'll need some information about the spectrum for the analysis, which we write into ```spectrum_params```. Then, we can find the $\ell$ values and the leakage spectra.

In [ ]:
spectrum_params = {'sky_decomp': [1,0,0], # IQU decomposition, usually [1,0,0]
                   'delta_ell': 50., # spacing between ell bins
                   'ell_max': 2500, # maximum ell value to analyze; keep at 2500 if using CMB_compare in next step
                   'choose_normalization': 'TT', # spectrum to normalize to, usually 'TT'
                   }

ell_abeam, spectra_abeam = mmt_tools.get_mmt_leakage(abeam, spectrum_params)

We can look at the leakage spectra on their own, or compare them to the CMB power spectra:

In [ ]:
mmt_tools.make_leakage_plot(ell_abeam, spectra_abeam, 'Analytic Beam with $e=0.001$', CMB_compare=False)

In [ ]:
mmt_tools.make_leakage_plot(ell_abeam, spectra_abeam, 'Analytic Beam with $e=0.001$', CMB_compare=True)

Feel free to change the ```analytic_beam_params``` and see what happens!

## HFSS Beam

Now that we know some of the basics of how the beams pipeline works, let's try it out with some feedhorn measurements! There are a lot more beam parameters here, because of how we need to interpret the data to make our beams, and it will take longer to run this process, but from the user side, it is very similar.

In [ ]:
hbeam_params = {'fmin': 125, # lower frequency edge of the band
                'fmax': 165, # upper frequency edge of the band
                'numfreqs': 40, # number of frequencies in between these to analyze
                'folder': {'I': 'hfss_iq/', # where you want to store the I & Q optimization
                           'U': 'hfss_u/', # where you want to store the U optimization
                          },
                'stop_angle': 13., # feedhorn stop angle
                'N': 1024, # number of pixels
                'mask_params': {'cen': 44.5, # mask center
                                'telecentricity': 0, # mask telecentricity
                                'savename': 'mask.txt', # filename to store the mask
                               },
                'center_freqs': [150], # center frequency
                'beam_fwhm': 1.5, # FWHM of the beam
                'pixel_size': 0.25/60., # pixel size in degrees
               }
hbeam_params['npix'] = hbeam_params['N']

hbeam = make_beams.HFSSBeam(hbeam_params) # this step is very verbose

```hbeam``` now holds all of the information we need about this beam. Just like before, we can make some plots of the beam leakage and make sure it looks the way we expect it to.

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(8,8))

qq = ax[0,0].imshow(hbeam.beam_matrix['QQ']) # Q->Q leakage
ax[0,0].set_title('QQ')
plt.colorbar(qq, ax=ax[0,0])

qi = ax[1,0].imshow(hbeam.beam_matrix['QI']) # Q->I leakage
ax[1,0].set_title('QI')
plt.colorbar(qi, ax=ax[1,0])

qu = ax[2,0].imshow(hbeam.beam_matrix['QU']) # Q->U leakage
ax[2,0].set_title('QU')
plt.colorbar(qu, ax=ax[2,0])

iq = ax[0,1].imshow(hbeam.beam_matrix['IQ']) # I->Q leakage
ax[0,1].set_title('IQ')

ii = ax[1,1].imshow(hbeam.beam_matrix['II']) # I->I leakage
ax[1,1].set_title('II')
plt.colorbar(ii, ax=ax[1,1])

iu = ax[2,1].imshow(hbeam.beam_matrix['IU']) # I->U leakage
ax[2,1].set_title('IU')

uq = ax[0,2].imshow(hbeam.beam_matrix['UQ']) # U->Q leakage
ax[0,2].set_title('UQ')

ui = ax[1,2].imshow(hbeam.beam_matrix['UI']) # U->I leakage
ax[1,2].set_title('UI')

uu = ax[2,2].imshow(hbeam.beam_matrix['UU']) # U->U leakage
ax[2,2].set_title('UU')
plt.colorbar(uu, ax=ax[2,2])

plt.show()

We would again like to see the leakage spectra. We'll use the same ```spectrum_params``` as before, for simplicity. Then, we can find the $\ell$ values and the leakage spectra.

In [ ]:
ell_hbeam, spectra_hbeam = mmt_tools.get_mmt_leakage(hbeam, spectrum_params)

We can look at the leakage spectra on their own, or compare them to the CMB power spectra:

In [ ]:
mmt_tools.make_leakage_plot(ell_hbeam, spectra_hbeam, 'Feedhorn Beam', CMB_compare=False)

In [ ]:
mmt_tools.make_leakage_plot(ell_hbeam, spectra_hbeam, 'Feedhorn Beam', CMB_compare=True)

## More things to do

Now that you know how to use the analytic and data-based pipeline, you can try changing some parameters to see how they affect the leakage spectra. Here are some things to try:

- Change the ellipticity in the analytic beam. Try values of 0.003, 0.01, and 0.03 and compare the leakage.
- Change the ```beam_fwhm``` parameters to 3.0 to see what a larger beam would look like.